# LangChain #
<ul>
    <ul>
        <li>LangChain is a framework the helps build applications around LLM's</li>
        <li>Its like a tool box for connecting LLM's with memory, API's, chains and agents</li>
        <li>The Fundamental bricks of Framework include LLM, PromptTemplate, Chains, Memory and Agents</li>
    </ul>
</ul>

## Brick 1---> <u>LLM</u> ##

<ul>
    <li>In LangChain an LLM Model is component that one can plug-in and swap</li>
</ul>

In [9]:
# Implementation of Simple LLM Call
from langchain_community.llms import Ollama


# Connecting to local llm mistral
llm = Ollama(
    model="mistral",
    temperature=0.2 # [0.3--> More deterministic and similar responses < 1.1 More Creative ]
    # For Factual and QA Based answering we want system to more reliable
    # So, in this context it is important to keep temperature low
)


# LLM Call
response = llm("Explain the need of LangGraph in one sentence when we already have LangChain")
print(response)

 LangGraph, unlike LangChain, focuses on providing a graph-based representation of language data, enabling more efficient and effective handling of complex relationships and patterns within the data, which can be particularly useful for tasks such as question answering, information extraction, and text summarization.


## Brick 2: PromptTemplate ##

<ul>
    <li>It is a reusable blue print of instruction we can give LLM's with {placeholders}</li>
    <li>Helps writing clean and dynamic content</li>
    <li>It is helpful when structure and task is same but content changes everytime</li>
</ul>
<p3>When to be careful</p3>
<ol>
    <li>If the task itself varies each time [One time Poem, another time SQL Query]</li>
</ol>

In [13]:
from langchain_community.llms import Ollama
from langchain.prompts import PromptTemplate

llm = Ollama(
    model="mistral",
    temperature=0.2
)


# We have to ensure that task remains same everytime before writing template

template = """
Explain the defination and purpose about the {concept} and give 3 fundamental usecase
"""

prompt = PromptTemplate(
    input_variables=["concept"],
    template=template
)


# Fill here the actual concept
final_prompt = prompt.format(concept="Artificial General Intellegence")

response = llm(final_prompt)

print(response)

 Artificial General Intelligence (AGI), also known as "strong AI," refers to a type of artificial intelligence that possesses the ability to understand, learn, and apply knowledge across a wide range of tasks at a level equal to or beyond human capability. Unlike Narrow AI, which is designed to perform specific tasks, AGI can reason, problem-solve, and think abstractly like humans.

The purpose of AGI is to create a machine that can perform any intellectual task that a human can do. This includes understanding complex concepts, learning from experience, making decisions under uncertainty, and even exhibiting creativity and consciousness. The ultimate goal is to build an AI system that can match or surpass human intelligence in its versatility and depth.

Here are three fundamental use cases of AGI:

1. Healthcare: AGI could revolutionize healthcare by analyzing vast amounts of medical data, diagnosing diseases more accurately than humans, and suggesting personalized treatment plans bas

## Brick 3: Chains ## 

<ul>
    <li>A Chain is a sequence of components[LLM's, Prompts, Tools, Memory etc..] linked together into one reusable pipeline</li>
    <li>Facilitates Flexibility and Reusability of Code</li>
    <li>We can define multiple components and use various combinations without writing the code of each combination every time seperatly</li>
</ul>

In [10]:
from langchain_community.llms import Ollama
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain


llm = Ollama(
    model="mistral",
    temperature=0.2
)

# Reusable Template
template = "Explain about the {topic} in one sentence"

prompt = PromptTemplate(
    input_varibales=["topic"],
    template=template
)

# Stacking of Multiple Components using Chains

chain = LLMChain(
    llm=llm,
    prompt=prompt
)

response = chain.run("AGI")
response2 = chain({"topic": "AGI"})

print(response)
print(type(response))

print(response2)
print(type(response2))

 Artificial General Intelligence (AGI) refers to a type of AI that has the ability to understand, learn, and apply knowledge across a wide range of tasks at a level equal to or beyond human capacity.
<class 'str'>
{'topic': 'AGI', 'text': ' Artificial General Intelligence (AGI) refers to a type of AI that has the ability to understand, learn, and apply knowledge across a wide range of tasks at a level equal to or beyond human capacity.'}
<class 'dict'>


<h4> Stacking of Multiple Chains </h4>
<ul>
    <li>Define Multiple Components and Chains and use different combinations</li>
    <li>Helps achieve code reusability</li>
</ul>
<p4>Below, is an example of 3 chains [Summarize---> Translate ----> Simplify]</p4>

In [17]:
from langchain_community.llms import Ollama
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain

llm = Ollama(
    model="mistral",
    temperature=0.2
)

# Step1: Summarize
summary_prompt = PromptTemplate(
    input_variables=["text"],
    template="Summarize the following in 2 Sentences \n\n: {text}"
)

summarize_chain = LLMChain(
    llm=llm,
    prompt=summary_prompt,
    output_key="summary" 
    # Each LLMChain produces output in the form of dictionary
    # "output_key" decides the dictionary key for that chain’s output so you can easily access it or pass it to the next chain.
)

# Step2: Translate to French Language
translate_prompt = PromptTemplate(
    input_variables=["summary"],
    template="Translate the following into French \n\n: {summary}",
)
translate_chain = LLMChain(
    llm=llm,
    prompt=translate_prompt,
    output_key="translation"
)

# Step3: Simplify in simple words
simplify_prompt = PromptTemplate(
    input_variables=["translation"],
    template="Explain this in simple French like for a 10 year old:\n\n{translation}"
)
simplify_chain = LLMChain(
    llm=llm,
    prompt=simplify_prompt,
    output_key="simple_french"
)

# Combine Into SequentialChain
overall_chain = SequentialChain(
    chains=[summarize_chain, translate_chain, simplify_chain],
    input_variables=["text"],
    output_variables=["summary", "translation", "simple_french"]
    # output_variables in the SequentialChain tells the overall chain 
    # which keys you want to keep in the final result
)

text =  """ Precision is one of the most important metric in machine learning. It measures the proportion of positive predictions that are
actually correct.It gives us an idea how confident the model is when it comes it predictions
"""

result = overall_chain({"text": text})
print(result)

{'text': ' Precision is one of the most important metric in machine learning. It measures the proportion of positive predictions that are\nactually correct.It gives us an idea how confident the model is when it comes it predictions\n', 'summary': " Precision in machine learning is a significant metric, indicating the ratio of correctly predicted positive instances to the total predicted positives. This measure helps gauge the confidence level of a model's predictions regarding positive outcomes.", 'translation': " La précision dans l'apprentissage automatique est un indicateur important, indiquant le rapport entre les instances positives correctement prédites et le total des instances positives prévues. Cette mesure permet d'évaluer le niveau de confiance du modèle quant aux résultats positifs prévus.\n\nEn français : La précision dans l'apprentissage automatique est un indicateur important, indiquant le rapport entre les instances positives correctement prédites et le total des instan

## Brick 4: Memory ##

<ul>
    <li>Memory allows the chain or agent to remember information across multiple calls</li>
    <li>Without memory every call is stateless, the model forgets everything after one prompt</li>
</ul>

<b>Types</b>
<ol>
    <li>ConversationalBufferMemory --> Keeps a running string of past conversations</li>
    <li>ConversationalSummarymemory --> Keeps a summarized content to save space</li>
    <li>Vector/Document memory ---> Stores embeddings for retrival [Used in RAG's]</li>
</ol>

<b>ConversationBufferMemory

In [24]:
from langchain_community.llms import Ollama
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory

llm = Ollama(
    model="mistral",
    temperature=0.2
)

# Initialize the memory
memory = ConversationBufferMemory(
    memory_key="chat_history", # Key to store the Chat
    return_messages = True # returns converstion history to LLM
)

prompt = PromptTemplate(
    input_variables=["user_input", "chat_history"],
    template=""" Act as an helpful assistannt and take {chat_history} as context
    and give answers to the {user_input}
    """
)

overall_chain = LLMChain(
    llm=llm,
    prompt=prompt,
    memory=memory
)

user_input = "What was the Percapita Income of India in 2001?"
response1 = overall_chain.run(user_input)
print(response1)

 The per capita income of India in the year 2001 was approximately $430 (USD). This value is subject to slight variations depending on the source, but this figure provides a general idea. For more accurate and up-to-date information, I recommend checking official sources such as the World Bank or the Indian government's statistical department.


In [26]:
response2 = overall_chain.run(user_input="Can you translate that in French?")
print(response2)

 Yes, I can translate that in French. The per capita income of India in the year 2001 was approximately 430 dollars (USD). This value is subject to slight variations depending on the source, but this figure provides a general idea. For more accurate and up-to-date information, I recommend checking official sources such as the World Bank or the Indian government's statistical department.

In French: Oui, je peux traduire ça en français. Le revenu par habitant de l'Inde en l'an 2001 était approximativement 430 dollars (USD). Cette valeur est sujette à des variations légères selon la source, mais cette chiffre fournit une idée générale. Pour des informations plus précises et à jour, je recommande de vérifier des sources officielles telles que la Banque mondiale ou le département statistique indien du gouvernement.


In [27]:
response3 = overall_chain.run(
    user_input="Give Per Capita Income in Rupees"
)

<b>ConversationSummaryMemory

In [33]:
from langchain_community.llms import Ollama
from langchain.prompts import PromptTemplate
from langchain.chains import ConversationChain
from langchain.memory import ConversationSummaryMemory

llm = Ollama(
    model="mistral",
    temperature=0.2
)

# Initialize the memory
memory = ConversationSummaryMemory(
    llm=llm,  # summary memory needs an llm to generate summaries
    memory_key="chat_history", # Key to store the Chat
    return_messages = True # returns converstion history to LLM
)

prompt = PromptTemplate(
    input_variables=["user_input", "chat_history"],
    template=""" Act as an helpful assistannt and take {chat_history} as context
    and give answers to the {user_input}
    """
)

overall_chain = LLMChain(
    llm=llm,
    prompt=prompt,
    memory=memory
)

user_input = "What was the Percapita Income of India in 2001?"
response1 = overall_chain.run(user_input)
print(response1)

 The per capita income of India in the year 2001 was approximately $430 (USD) according to World Bank data. However, it's important to note that inflation and exchange rates can affect these figures over time, so for the most accurate and up-to-date information, I recommend visiting a reliable source such as the World Bank or the Indian government's official statistics portal.


In [35]:
response2 = overall_chain({"user_input": "How much did it increase in 2011?"})
print(response2)

{'user_input': 'How much did it increase in 2011?', 'chat_history': [SystemMessage(content=" The human asks about the Per Capita Income of India in 2001. The AI responds that it was approximately $430 (USD) according to World Bank data, but notes that inflation and exchange rates can affect these figures over time, suggesting visiting a reliable source like the World Bank or Indian government's official statistics portal for accurate and up-to-date information.", additional_kwargs={}, response_metadata={})], 'text': " According to World Bank data, the per capita income of India increased significantly from approximately $430 (USD) in 2001 to around $1,057 (USD) in 2011. However, it's important to note that inflation and exchange rates can affect these figures over time. For accurate and up-to-date information, I recommend visiting a reliable source like the World Bank or Indian government's official statistics portal."}


<h4><b></b>VectorStoreRetreivereMemory [RAG]</h4>

In [1]:
import os
import pandas as pd
import pdfplumber
from pathlib import Path

BASE_DIR = Path().resolve().parent
DATA_DIR = BASE_DIR / "data"


def extract_chunks_from_page(text, filename, page_num, chunk_size=300, overlap=50):
    """
    PdfPlumber extracts the text of a one page at a time.
    To extract the text of all the pages we have to run a loop, where in each iteration the text of single page will be removed.
    After removal of text this function will be called, which takes the text as the input of a single page and creates chunks.
    {A Chunk is a fixed sized segment of a text document from a consective text split}
    """
    # The text is split into a list of words to tokenize
    words = text.strip().split()
    chunks=[]
    # Here chunks will be created iteratively
    # Each chunk will have 300 words with a 50 word overlap
    for word in range(0, len(words), chunk_size-overlap):
        chunk_words = words[word:word+chunk_size]
        if len(chunk_words)<30: # Skip if the length of the chunk words is less than 30
            continue
        chunk_text = " ".join(chunk_words) # This helps combining all the list of words into a single string
        chunk_id = f"{filename}_p{page_num}_c{word}"
        # Will create a dictionary that returns meta data and chunked text of each page
        chunks.append(
            {
                "filename": filename,
                "page": page_num,
                "Retrieval_ID": chunk_id,
                "text": chunk_text
            }
        ) # This will return a list of dictionaries
    return chunks

In [2]:
def extract_chunks_from_pdf(DATA_DIR):
    # Accessing all the filenames from the directory
    all_chunks=[]
    pdf_files = [filename for filename in os.listdir(DATA_DIR) if filename.endswith(".pdf")]
    for filename in pdf_files:
        pdf_path = os.path.join(DATA_DIR, filename)
        with pdfplumber.open(pdf_path) as pdf:
            for page_num, page in enumerate(pdf.pages, start=1):
                text = page.extract_text()
                if not text:
                    continue
                text = text.replace("/n", "").strip()
                chunks = extract_chunks_from_page(text, filename, page_num)
                all_chunks.extend(chunks)
    return all_chunks


if __name__ == "__main__":
    chunks = extract_chunks_from_pdf(DATA_DIR)
    df = pd.DataFrame(chunks)
    df.head()

In [3]:
df.head()

,filename,page,Retrieval_ID,text
0,HDFC_AGM_Transcript_Aug2024.pdf,1,HDFC_AGM_Transcript_Aug2024.pdf_p1_c0,"CIN: L65920MH1994PLC080618 HDFC Bank Limited, ..."
1,HDFC_AGM_Transcript_Aug2024.pdf,1,HDFC_AGM_Transcript_Aug2024.pdf_p1_c250,by way of remote e-voting will not be able to ...
2,HDFC_AGM_Transcript_Aug2024.pdf,2,HDFC_AGM_Transcript_Aug2024.pdf_p2_c0,"resolutions as read. Now, without any further ..."
3,HDFC_AGM_Transcript_Aug2024.pdf,2,HDFC_AGM_Transcript_Aug2024.pdf_p2_c250,"present. With permission of the members, I cal..."
4,HDFC_AGM_Transcript_Aug2024.pdf,3,HDFC_AGM_Transcript_Aug2024.pdf_p3_c0,"Friends, we would like to recall last year whe..."


In [25]:
from langchain_community.llms import Ollama
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, ConversationChain
from langchain.memory import ConversationSummaryMemory
from langchain.vectorstores import Chroma
from langchain.schema import Document
from langchain.embeddings import HuggingFaceEmbeddings


# Loading the LLM Model {Brick1}
llm = Ollama(
    model="mistral",
    temperature=0.3)


# Creating a Template {Brick2}
prompt = PromptTemplate(
    input_variables = ["chat_history", "context", "user_input"],
    template = """
    Act as a Financial QA Assistant and provide structural insights using the 
    {context} and the {chat_history} to the {user_input}
    """
)

# Initializing the Memory {Brick4}
memory = ConversationSummaryMemory(
    llm=llm,
    memory_key="chat_history",
    return_messages=True,
    input_key="user_input"
)

# Chaining all the components
chain = LLMChain(
    llm=llm,
    prompt=prompt,
    memory=memory
)

# A Document is a tiny container LangChain uses to hold a chunk of text plus optional metadata.
documents = [
    Document(page_content=row["text"], metadata={"filename":row["filename"],
                                                 "page":row["page"],
                                                 "Retrieval_ID":row["Retrieval_ID"]})
    for index, row in df.iterrows()
]

embedding_transformer = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vector_storage = Chroma.from_documents(documents, embedding_transformer)
retriever = vector_storage.as_retriever(search_kwargs={"k":3})

In [31]:
# Preparaion of input_variables ["user_input", "context", "chat_history"]

user_input = "Explain the brief summary of the document"
chat_history = memory.load_memory_variables({})["chat_history"]
docs = retriever.get_relevant_documents(user_input)
context = "/n".join(
    [x.page_content for x in docs]
)
chain.run(user_input=user_input, context=context)

' The document appears to be a transcript from a meeting or presentation, possibly related to the annual report of HDFC Bank. The speaker, Mr. Atanu Chakraborty, is discussing the challenges faced by banking systems and corporates regarding mobile numbers, and how their company is addressing these issues with their best efforts. He also mentions that detailed information about specific items such as audit fees, legal and professional fees can be found in the Annual Report, which consists of over 500 pages. The speaker expresses gratitude for the patience of the audience during the presentation and thanks them for their attention. However, the document does not provide a brief summary of the document itself.'

In [33]:
user_input2 = "Can you translate that in french?"
chain.run(user_input=user_input2, context=context)

" Le document en question est une transcription d'une réunion ou présentation liée à l'annuel de HDFC Bank, où M. Atanu Chakraborty discute des défis liés aux numéros de mobile rencontrés par les systèmes bancaires et les entreprises, ainsi que comment HDFC Bank y travaille pour résoudre ces problèmes. Il mentionne également que les informations détaillées sur certains éléments spécifiques comme les frais d'audit et les frais de droit et professionnels peuvent être trouvés dans différentes pages du rapport annuel, qui compte plus de 500 pages. Le locuteur remercie l'audience pour leur patience et attention, mais le document ne fournit pas une résumé court de lui-même.\n\n(Traduction automatique)"

In [ ]:
import os
import pdfplumber
import pandas as pd
from langchain_community.llms import Ollama
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.memory import ConversationSummaryMemory
from langchain.vectorstores import Chroma
from langchain.schema import Document
from langchain.embeddings import HuggingFaceEmbeddings
from pathlib import Path

BASE_DIR = Path().resolve().parent
DATA_DIR = BASE_DIR / "data"


class DocumentProcessor:
    def __init__(self, DATA_DIR, chunk_size=300, overlap=50, min_chunk_len=30):
        self.data_dir = DATA_DIR
        self.chunk_size = chunk_size
        self.overlap = overlap
        self.min_chunk_len = min_chunk_len

    def _extract_chunks_from_page(self, text, filename, page_num):
        words = text.strip().split()
        chunks = []
        for word in range(0, len(words), self.chunk_size - self.overlap):
            chunk_words = words[word:word + self.chunk_size]
            if len(chunk_words) < self.min_chunk_len:
                continue
            chunk_text = " ".join(chunk_words)
            chunk_id = f"{filename}_p{page_num}_c{word}"
            chunks.append(
                {
                    "filename": filename,
                    "page": page_num,
                    "Retrieval_ID": chunk_id,
                    "text": chunk_text
                }
            )
        return chunks

    def extract_from_pdfs(self):
        all_chunks = []
        pdf_files = [f for f in os.listdir(self.data_dir) if f.endswith(".pdf")]

        for filename in pdf_files:
            pdf_path = os.path.join(self.data_dir, filename)
            with pdfplumber.open(pdf_path) as pdf:
                for page_num, page in enumerate(pdf.pages, start=1):
                    text = page.extract_text()
                    if not text:
                        continue
                    text = text.replace("\n", " ").strip()
                    chunks = self._extract_chunks_from_page(text, filename, page_num)
                    all_chunks.extend(chunks)

        return pd.DataFrame(all_chunks)


class VectorStoreManager:
    def __init__(self, model_name="all-MiniLM-L6-v2"):
        self.embedding_transformer = HuggingFaceEmbeddings(model_name=model_name)
        self.vector_storage = None

    def build_store(self, df):
        documents = [
            Document(
                page_content=row["text"],
                metadata={
                    "filename": row["filename"],
                    "page": row["page"],
                    "Retrieval_ID": row["Retrieval_ID"]
                }
            )
            for index, row in df.iterrows()
        ]
        self.vector_storage = Chroma.from_documents(documents, self.embedding_transformer)
        return self.vector_storage

    def get_retriever(self, k=3):
        if not self.vector_storage:
            raise ValueError("Vector storage has not been initialized. Call build_store() first.")
        return self.vector_storage.as_retriever(search_kwargs={"k": k})


class FinancialQA_Assistant:
    def __init__(self, model="mistral", temperature=0.3):
        self.llm = Ollama(model=model, temperature=temperature)
        self.memory = ConversationSummaryMemory(
            llm=self.llm,
            memory_key="chat_history",
            return_messages=True,
            input_key="user_input"
        )
        self.prompt = PromptTemplate(
            input_variables=["chat_history", "context", "user_input"],
            template="""
            Act as a Financial QA Assistant and provide structural insights using 
            the {context} and the {chat_history} to answer: {user_input}
            """
        )
        self.chain = LLMChain(llm=self.llm, prompt=self.prompt, memory=self.memory)

    def run(self, user_input, retriever):
        chat_history = self.memory.load_memory_variables({})["chat_history"]
        docs = retriever.get_relevant_documents(user_input)
        context = "\n".join([doc.page_content for doc in docs])
        return self.chain.run(user_input=user_input, context=context)


class FinancialQAPipeline:
    def __init__(self, data_dir):
        self.processor = DocumentProcessor(data_dir)
        self.vector_manager = VectorStoreManager()
        self.assistant = FinancialQA_Assistant()

    def run(self, user_query):
        df = self.processor.extract_from_pdfs()
        self.vector_manager.build_store(df)
        retriever = self.vector_manager.get_retriever(k=3)
        return self.assistant.run(user_query, retriever)


if __name__ == "__main__":
    pipeline = FinancialQAPipeline(DATA_DIR)
    answer = pipeline.run("Explain the brief summary of the document")
    print(answer)
